In [1]:
import json
import pandas as pd
import statistics as stats
import re
import pprint

In [203]:
ALL_TABULAR_DATASETS = ["adult", "census", "child", "covtype", "credit", "insurance", "health_insurance", 
                       "intrusion", "drugs", "loan", "pums"]
ALL_TABULAR_MODELS = {
    "gretel": ["actgan"], 
    "sdv": ["ctgan", "tvae", "gaussian_copula"], 
    "syn": ["ctgan", "tvae", "goggle", "arf", "ddpm", "nflow", "rtvae"], 
    "llm": ["great"], 
    "betterdata": ["gan", "gan_dp"]
}

ALL_SEQUENTIAL_DATASETS = ["taxi", "nasdaq", "pums"]

ALL_SEQUENTIAL_MODELS = {
    "gretel": ["dgan"],
    "sdv": ["par"]
}

In [11]:
def get_cmds_configs(command_string):

    # Sample command string
    # command_string = "python3 run_model.py --m sequential --l sdv --s par --data nasdaq --o outputs --run_model_training --use_gpu"

    # Regular expression pattern to extract arguments for "--l", "--s", and "--data"
    pattern = r"--l (\S+).*?--s (\S+).*?--data (\S+)"

    # Use regex to find the match
    match = re.search(pattern, command_string)

    # Extract values from the match
    library = match.group(1) if match else None
    synthesizer = match.group(2) if match else None
    data = match.group(3) if match else None

    return (library, synthesizer, data)

In [12]:
sdg_jobs_df = pd.read_csv("../final_outs/sdg_jobs.csv")
if 'Unnamed: 0' in sdg_jobs_df.columns:
    sdg_jobs_df.drop(columns=['Unnamed: 0'], inplace=True)

In [223]:
sdg_jobs_df.columns

Index(['Experiment Name', 'Run ID', 'Status', 'Creation Time', 'End Time',
       'Duration (seconds)', 'Compute Target', 'Command Parameters'],
      dtype='object')

In [225]:
# sdg_jobs_df[["Command Parameters"]]

In [13]:
# Iterate over rows and print 'Command Parameters' for specific 'Status' values
TABULAR_INCOMPLETE_JOBS = {
    "gretel": {}, 
    "sdv": {}, 
    "syn": {}
}
for index, row in sdg_jobs_df.iterrows():
    if row['Status'] in ['Failed', 'Canceled', 'Running']:
        (library, synthesizer, data) = get_cmds_configs(row['Command Parameters'])
        if library and synthesizer and data:
            # print(library, synthesizer, data)
            
            if library == "synthcity":
                library = "syn"
            
            if synthesizer in TABULAR_INCOMPLETE_JOBS[library]:
                TABULAR_INCOMPLETE_JOBS[library][synthesizer][data] = row['Status']
            else:
                TABULAR_INCOMPLETE_JOBS[library][synthesizer] = {}
                TABULAR_INCOMPLETE_JOBS[library][synthesizer][data] = row['Status']
                
                
pprint.pprint(TABULAR_INCOMPLETE_JOBS)

{'gretel': {'dgan': {'pums': 'Failed'}},
 'sdv': {'ctgan': {'insurance': 'Failed'}, 'par': {'pums': 'Failed'}},
 'syn': {'arf': {'pums': 'Failed'},
         'ctgan': {'covtype': 'Running',
                   'credit': 'Canceled',
                   'pums': 'Failed'},
         'ddpm': {'pums': 'Canceled'},
         'goggle': {'adult': 'Failed',
                    'covtype': 'Failed',
                    'intrusion': 'Failed',
                    'loan': 'Failed',
                    'pums': 'Running'},
         'nflow': {'adult': 'Failed',
                   'covtype': 'Failed',
                   'loan': 'Failed',
                   'pums': 'Failed'},
         'rtvae': {'credit': 'Canceled', 'pums': 'Failed'},
         'tvae': {'pums': 'Failed'}}}


In [246]:
ERROR_VAL  = -99
def get_scores_df(exp_dataset, models_list, case="tabular"):
    
    final_report = {
        "dataset": [], 
        "model": [],
        "case": [], 
        "domain_cov": [], 
        "stats_cov": [], 
        "outliers_cov": [],
        "missing_cov": [],
        "ks_sim": [], 
        "tv_sim": [],
        "corr_sim": [], 
        "contin_sim": [], 
        "sdv_quality_report": [],
        "wass_dist": [], 
        "js_dist": [], 
        "new_row_synthesis": []
    }
    


    for lib, models in models_list.items():
        
        if case == "hpo" and lib!= "syn":
            continue
        for model in models: 
            
            if case in ["hyperimpute", "ice", "missforest", "simple", "only_hyperimpute", "only_ice", "only_missforest", "only_simple",]:  
                corr_file = f"../metrics_out/hyperimpute/{model}_{lib}/{exp_dataset}/{case}/{exp_dataset}_{model}_correlation.csv"
                metrics_file = f"../metrics_out/hyperimpute/{model}_{lib}/{exp_dataset}/{case}/{exp_dataset}_{model}_metrics.json"
            else:
                corr_file = f"../metrics_out/{case}/{model}_{lib}/{exp_dataset}/{exp_dataset}_{model}_correlation.csv"
                metrics_file = f"../metrics_out/{case}/{model}_{lib}/{exp_dataset}/{exp_dataset}_{model}_metrics.json"
            
            # metrics_out/hyperimpute/ctgan/drugs/only_hyperimpute/drugs_ctgan_correlation.csv
            # metrics_out/hyperimpute/ctgan/drugs/hyperimpute/drugs_ctgan_correlation.csv
            
            # /Users/anshusingh/DPPCC/whitespace/benchmarking-synthetic-data-generators/metrics_out/hyperimpute/ctgan_sdv/drugs/ice/drugs_ctgan_correlation.csv
                        
            try:
                with open(metrics_file, 'r') as file:
                    metrics_scores = json.load(file)
                corr_df = pd.read_csv(corr_file)
                print("Success:", metrics_file)
            except Exception as e:
                print(e)
                # print(model, lib)
                # status = None
                # if lib in TABULAR_INCOMPLETE_JOBS and model in TABULAR_INCOMPLETE_JOBS[lib]:
                #     if exp_dataset in TABULAR_INCOMPLETE_JOBS[lib][model]:
                #         # print("Status: ",  TABULAR_INCOMPLETE_JOBS[lib][model][exp_dataset])
                #         # print("-")
                #         status = TABULAR_INCOMPLETE_JOBS[lib][model][exp_dataset]
                #     else:
                #         status = -99 #"Skipped"
                # else:
                #     status =  -99 #"Skipped"
                #     # print(e)
                #     # print("Skipped")
                # for k in final_report.keys():
                #     if k not in ["model", "dataset", "case"]: 
                        # final_report[k].append(-99) #.append(status)
                    # print(final_report)
                continue
            
            final_report["model"].append(f"{model}_{lib}")
            final_report["dataset"].append(exp_dataset)
            final_report["case"].append(case)
            
            # print(final_report) 
            
            if metrics_scores["coverage"]["domain_coverage"].values():
                domain_coverage_mean = round(stats.mean(metrics_scores["coverage"]["domain_coverage"].values()), 2)
                final_report["domain_cov"].append(domain_coverage_mean)
            else:
                # Handle the case for empty list, e.g., append a default value or skip
                final_report["domain_cov"].append(ERROR_VAL)  # or any other default value
            
            
            if metrics_scores["coverage"]["missing_values_coverage"].values():
                missing_values_coverage_mean = round(stats.mean(metrics_scores["coverage"]["missing_values_coverage"].values()), 2)
                final_report["missing_cov"].append(missing_values_coverage_mean)
            else:
                # Handle the case for empty list, e.g., append a default value or skip
                final_report["missing_cov"].append(ERROR_VAL)  # or any other default value
            
            
            if metrics_scores["coverage"]["outlier_coverage"].values():
                outlier_coverage_mean = round(stats.mean(metrics_scores["coverage"]["outlier_coverage"].values()), 2)
                final_report["outliers_cov"].append(outlier_coverage_mean)
            else:
                # Handle the case for empty list, e.g., append a default value or skip
                final_report["outliers_cov"].append(ERROR_VAL)  # or any other default value

            # final_report["outliers_cov"].append(round(stats.mean(metrics_scores["coverage"]["outlier_coverage"].values()), 2))

            # Initialize total and count variables
            total_average = 0
            count = 0
            # Calculate the average of mean, median, and std for each column
            try:
                for c, s in metrics_scores["similarity"]["statistic"].items():
                    column_average = (s["mean"] + s["median"] + s["std"]) / 3
                    total_average += column_average
                    count += 1
                # Calculate the overall average
                overall_average = total_average / count if count > 0 else ERROR_VAL
                final_report["stats_cov"].append(round(overall_average, 2))
            except:
                final_report["stats_cov"].append(ERROR_VAL)

            ks_scores = []
            tv_scores = []
            for key, value in metrics_scores["sdv_quality_report"]["distribution"].items():
                if value["metric"] == "KSComplement":
                    ks_scores.append(value["score"])
                elif value["metric"] == "TVComplement":
                    tv_scores.append(value["score"])

            # Calculating averages
            ks_average = stats.mean(ks_scores) if ks_scores else ERROR_VAL
            tv_average = stats.mean(tv_scores) if tv_scores else ERROR_VAL
            final_report["ks_sim"].append(round(ks_average, 2))
            final_report["tv_sim"].append(round(tv_average, 2))


            # contingency_average, correlation_average
            contingency_average = corr_df[corr_df['Metric'] == 'ContingencySimilarity']["Score"].mean()
            correlation_average = corr_df[corr_df['Metric'] == 'CorrelationSimilarity']["Score"].mean()

            final_report["contin_sim"].append(round(contingency_average, 2))
            final_report["corr_sim"].append(round(correlation_average, 2))

            final_report["sdv_quality_report"].append(round(metrics_scores["sdv_quality_report"]["score"], 2))
            
            if metrics_scores["similarity"]["js_distance"].values():
                js_dist_mean = round(stats.mean(metrics_scores["similarity"]["js_distance"].values()), 2)
                final_report["js_dist"].append(js_dist_mean)
            else:
                # Handle the case for empty list, e.g., append a default value or skip
                final_report["js_dist"].append(ERROR_VAL)  # or any other default value
                
            if metrics_scores["similarity"]["wass_distance"].values():
                wass_dist_mean = round(stats.mean(metrics_scores["similarity"]["wass_distance"].values()), 2)
                final_report["wass_dist"].append(wass_dist_mean)
            else:
                # Handle the case for empty list, e.g., append a default value or skip
                final_report["wass_dist"].append(ERROR_VAL)  # or any other default value

            # # final_report["js_dist"].append(round(stats.mean(metrics_scores["similarity"]["js_distance"].values()), 2))
            # final_report["wass_dist"].append(round(stats.mean(metrics_scores["similarity"]["wass_distance"].values()), 2))
            
            if "score" in metrics_scores["privacy"]["new_row_synthesis"]:
                final_report["new_row_synthesis"].append(round(metrics_scores["privacy"]["new_row_synthesis"]["score"], 2))
            else:
                final_report["new_row_synthesis"].append(ERROR_VAL)
            
    return final_report

In [248]:
ALL_HYPERIMPUTE_MODELS = {
    "sdv": ["ctgan"]
}

IMPUTERS = ["missforest", "ice", "hyperimpute", "simple"]

tabular_bm_df = pd.DataFrame()
for imputer in IMPUTERS:
    final_report = get_scores_df("drugs", ALL_HYPERIMPUTE_MODELS, imputer)
    tabular_bm_df = tabular_bm_df.append(pd.DataFrame(final_report), ignore_index=True)
    
    final_report = get_scores_df("drugs", ALL_HYPERIMPUTE_MODELS, "only_" + imputer)
    tabular_bm_df = tabular_bm_df.append(pd.DataFrame(final_report), ignore_index=True)
    
# for imputer in IMPUTERS:
#     pd.DataFrame(get_scores_df("drugs", ALL_HYPERIMPUTE_MODELS, "only_"+ imputer))
    
tabular_bm_df

# /Users/anshusingh/DPPCC/whitespace/benchmarking-synthetic-data-generators/metrics_out/hyperimpute/ctgan_sdv/drugs/ice/drugs_ctgan_metrics.json

Success: ../metrics_out/hyperimpute/ctgan_sdv/drugs/missforest/drugs_ctgan_metrics.json
Success: ../metrics_out/hyperimpute/ctgan_sdv/drugs/only_missforest/drugs_ctgan_metrics.json
Success: ../metrics_out/hyperimpute/ctgan_sdv/drugs/ice/drugs_ctgan_metrics.json
Success: ../metrics_out/hyperimpute/ctgan_sdv/drugs/only_ice/drugs_ctgan_metrics.json
Success: ../metrics_out/hyperimpute/ctgan_sdv/drugs/hyperimpute/drugs_ctgan_metrics.json
Success: ../metrics_out/hyperimpute/ctgan_sdv/drugs/only_hyperimpute/drugs_ctgan_metrics.json
Success: ../metrics_out/hyperimpute/ctgan_sdv/drugs/simple/drugs_ctgan_metrics.json
Success: ../metrics_out/hyperimpute/ctgan_sdv/drugs/only_simple/drugs_ctgan_metrics.json


/var/folders/3l/sc82d46d2_lbr0vhyw2tf0t80000gn/T/ipykernel_40693/3218083429.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabular_bm_df = tabular_bm_df.append(pd.DataFrame(final_report), ignore_index=True)
/var/folders/3l/sc82d46d2_lbr0vhyw2tf0t80000gn/T/ipykernel_40693/3218083429.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabular_bm_df = tabular_bm_df.append(pd.DataFrame(final_report), ignore_index=True)
/var/folders/3l/sc82d46d2_lbr0vhyw2tf0t80000gn/T/ipykernel_40693/3218083429.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabular_bm_df = tabular_bm_df.append(pd.DataFrame(final_report), ignore_index=True)
/var/folders/3l/sc82d46d2_lbr0vhyw2tf0t80000gn/T/ipykernel_40693/3218083429.py:13: FutureWarni

,dataset,model,case,domain_cov,stats_cov,outliers_cov,missing_cov,ks_sim,tv_sim,corr_sim,contin_sim,sdv_quality_report,wass_dist,js_dist,new_row_synthesis
0,drugs,ctgan_sdv,missforest,0.92,0.99,0.0,0.42,0.95,0.88,0.92,0.17,0.53,0.01,0.07,-99
1,drugs,ctgan_sdv,only_missforest,0.93,1.00,1.0,0.42,0.98,0.91,0.97,0.19,0.55,0.01,0.05,-99
2,drugs,ctgan_sdv,ice,0.96,0.93,0.0,0.42,0.90,0.89,0.95,0.17,0.53,0.09,0.22,-99
3,drugs,ctgan_sdv,only_ice,0.96,1.00,1.0,0.42,0.98,0.92,0.97,0.19,0.56,0.01,0.05,-99
4,drugs,ctgan_sdv,hyperimpute,0.95,0.92,1.0,0.42,0.85,0.90,0.96,0.17,0.53,0.12,0.27,-99
5,drugs,ctgan_sdv,only_hyperimpute,0.95,1.00,1.0,0.42,0.98,0.92,0.97,0.19,0.56,0.01,0.05,-99
6,drugs,ctgan_sdv,simple,0.90,0.98,0.0,0.42,0.94,0.89,0.96,0.17,0.53,0.03,0.10,-99
7,drugs,ctgan_sdv,only_simple,0.91,1.00,1.0,0.42,0.98,0.92,0.97,0.19,0.56,0.01,0.05,-99


In [222]:
# pd.DataFrame( get_scores_df("nasdaq", ALL_SEQUENTIAL_MODELS, "sequential"))
pd.DataFrame( get_scores_df("pums", ALL_SEQUENTIAL_MODELS, "sequential"))

# /Users/anshusingh/DPPCC/whitespace/benchmarking-synthetic-data-generators/metrics_out/sequential/dgan/nasdaq/nasdaq_dgan_correlation.csv

[Errno 2] No such file or directory: '../metrics_out/sequential/dgan_gretel/pums/pums_dgan_metrics.json'
[Errno 2] No such file or directory: '../metrics_out/sequential/par_sdv/pums/pums_par_metrics.json'


,dataset,model,case,domain_cov,stats_cov,outliers_cov,missing_cov,ks_sim,tv_sim,corr_sim,contin_sim,sdv_quality_report,wass_dist,js_dist,new_row_synthesis


In [207]:
pd.DataFrame( get_scores_df("loan", ALL_TABULAR_MODELS, "hpo"))

[Errno 2] No such file or directory: '../metrics_out/hpo/goggle_syn/loan/loan_goggle_metrics.json'
[Errno 2] No such file or directory: '../metrics_out/hpo/nflow_syn/loan/loan_nflow_metrics.json'


,dataset,model,case,domain_cov,stats_cov,outliers_cov,missing_cov,ks_sim,tv_sim,corr_sim,contin_sim,sdv_quality_report,wass_dist,js_dist,new_row_synthesis
0,loan,ctgan_syn,hpo,0.97,0.99,0.40,1.0,0.91,0.98,0.96,0.93,0.94,0.03,0.17,1.0
1,loan,tvae_syn,hpo,0.95,0.98,0.35,1.0,0.93,0.98,0.96,0.92,0.94,0.02,0.19,1.0
2,loan,arf_syn,hpo,1.00,0.99,0.46,1.0,0.90,0.99,0.98,0.93,0.94,0.02,0.14,1.0
3,loan,ddpm_syn,hpo,1.00,0.94,0.45,1.0,0.95,0.97,0.98,0.88,0.93,0.03,0.10,1.0
4,loan,rtvae_syn,hpo,0.87,0.97,0.35,1.0,0.87,0.98,0.94,0.81,0.88,0.04,0.47,1.0


In [208]:
# exp_dataset = "adult"
ALL_TABULAR_DATASETS = ["adult", "census", "child", "covtype", "credit", "insurance", "health_insurance", 
                       "intrusion", "drugs", "loan", "pums"]

tabular_bm_df = pd.DataFrame()
for exp_dataset in ALL_TABULAR_DATASETS:
    final_report = get_scores_df(exp_dataset, ALL_TABULAR_MODELS)
    tabular_bm_df = tabular_bm_df.append(pd.DataFrame(final_report), ignore_index=True)

    
    # try:
    #     pd.DataFrame(final_report)
    # except Exception as e:
    #     print(e)
    #     print("~"*10, exp_dataset)
    #     for k, v in final_report.items():
    #         print(k, len(v))

/var/folders/3l/sc82d46d2_lbr0vhyw2tf0t80000gn/T/ipykernel_40693/544052681.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabular_bm_df = tabular_bm_df.append(pd.DataFrame(final_report), ignore_index=True)
/var/folders/3l/sc82d46d2_lbr0vhyw2tf0t80000gn/T/ipykernel_40693/544052681.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabular_bm_df = tabular_bm_df.append(pd.DataFrame(final_report), ignore_index=True)


[Errno 2] No such file or directory: '../metrics_out/tabular/ctgan_sdv/census/census_ctgan_metrics.json'
[Errno 2] No such file or directory: '../metrics_out/tabular/ctgan_syn/census/census_ctgan_metrics.json'
[Errno 2] No such file or directory: '../metrics_out/tabular/tvae_syn/census/census_tvae_metrics.json'
[Errno 2] No such file or directory: '../metrics_out/tabular/goggle_syn/census/census_goggle_metrics.json'
[Errno 2] No such file or directory: '../metrics_out/tabular/arf_syn/census/census_arf_metrics.json'
[Errno 2] No such file or directory: '../metrics_out/tabular/ddpm_syn/census/census_ddpm_metrics.json'
[Errno 2] No such file or directory: '../metrics_out/tabular/nflow_syn/census/census_nflow_metrics.json'
[Errno 2] No such file or directory: '../metrics_out/tabular/rtvae_syn/census/census_rtvae_metrics.json'
[Errno 2] No such file or directory: '../metrics_out/tabular/great_llm/census/census_great_metrics.json'
[Errno 2] No such file or directory: '../metrics_out/tabular/

/var/folders/3l/sc82d46d2_lbr0vhyw2tf0t80000gn/T/ipykernel_40693/544052681.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabular_bm_df = tabular_bm_df.append(pd.DataFrame(final_report), ignore_index=True)


[Errno 2] No such file or directory: '../metrics_out/tabular/nflow_syn/covtype/covtype_nflow_metrics.json'
[Errno 2] No such file or directory: '../metrics_out/tabular/great_llm/covtype/covtype_great_metrics.json'
[Errno 2] No such file or directory: '../metrics_out/tabular/gan_betterdata/covtype/covtype_gan_metrics.json'
[Errno 2] No such file or directory: '../metrics_out/tabular/gan_dp_betterdata/covtype/covtype_gan_dp_metrics.json'


/var/folders/3l/sc82d46d2_lbr0vhyw2tf0t80000gn/T/ipykernel_40693/544052681.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabular_bm_df = tabular_bm_df.append(pd.DataFrame(final_report), ignore_index=True)


...

/var/folders/3l/sc82d46d2_lbr0vhyw2tf0t80000gn/T/ipykernel_40693/544052681.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabular_bm_df = tabular_bm_df.append(pd.DataFrame(final_report), ignore_index=True)


[Errno 2] No such file or directory: '../metrics_out/tabular/great_llm/drugs/drugs_great_metrics.json'
[Errno 2] No such file or directory: '../metrics_out/tabular/gan_betterdata/drugs/drugs_gan_metrics.json'
[Errno 2] No such file or directory: '../metrics_out/tabular/gan_dp_betterdata/drugs/drugs_gan_dp_metrics.json'


/var/folders/3l/sc82d46d2_lbr0vhyw2tf0t80000gn/T/ipykernel_40693/544052681.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabular_bm_df = tabular_bm_df.append(pd.DataFrame(final_report), ignore_index=True)


[Errno 2] No such file or directory: '../metrics_out/tabular/ctgan_syn/pums/pums_ctgan_metrics.json'
[Errno 2] No such file or directory: '../metrics_out/tabular/tvae_syn/pums/pums_tvae_metrics.json'
[Errno 2] No such file or directory: '../metrics_out/tabular/goggle_syn/pums/pums_goggle_metrics.json'
[Errno 2] No such file or directory: '../metrics_out/tabular/arf_syn/pums/pums_arf_metrics.json'
[Errno 2] No such file or directory: '../metrics_out/tabular/ddpm_syn/pums/pums_ddpm_metrics.json'
[Errno 2] No such file or directory: '../metrics_out/tabular/nflow_syn/pums/pums_nflow_metrics.json'
[Errno 2] No such file or directory: '../metrics_out/tabular/rtvae_syn/pums/pums_rtvae_metrics.json'
[Errno 2] No such file or directory: '../metrics_out/tabular/great_llm/pums/pums_great_metrics.json'
[Errno 2] No such file or directory: '../metrics_out/tabular/gan_betterdata/pums/pums_gan_metrics.json'
[Errno 2] No such file or directory: '../metrics_out/tabular/gan_dp_betterdata/pums/pums_gan_d

/var/folders/3l/sc82d46d2_lbr0vhyw2tf0t80000gn/T/ipykernel_40693/544052681.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabular_bm_df = tabular_bm_df.append(pd.DataFrame(final_report), ignore_index=True)
/var/folders/3l/sc82d46d2_lbr0vhyw2tf0t80000gn/T/ipykernel_40693/544052681.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabular_bm_df = tabular_bm_df.append(pd.DataFrame(final_report), ignore_index=True)


In [209]:
tabular_bm_df.shape

(103, 15)

In [211]:
# exp_dataset = "adult"
HPO_DATASETS = ["adult", "loan"]
# tabular_bm_df = pd.DataFrame()
for exp_dataset in HPO_DATASETS:
    final_report = get_scores_df(exp_dataset, ALL_TABULAR_MODELS, "hpo")
    tabular_bm_df = tabular_bm_df.append(pd.DataFrame(final_report), ignore_index=True)

[Errno 2] No such file or directory: '../metrics_out/hpo/goggle_syn/adult/adult_goggle_metrics.json'
[Errno 2] No such file or directory: '../metrics_out/hpo/nflow_syn/adult/adult_nflow_metrics.json'
[Errno 2] No such file or directory: '../metrics_out/hpo/goggle_syn/loan/loan_goggle_metrics.json'
[Errno 2] No such file or directory: '../metrics_out/hpo/nflow_syn/loan/loan_nflow_metrics.json'


/var/folders/3l/sc82d46d2_lbr0vhyw2tf0t80000gn/T/ipykernel_40693/4110285772.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabular_bm_df = tabular_bm_df.append(pd.DataFrame(final_report), ignore_index=True)
/var/folders/3l/sc82d46d2_lbr0vhyw2tf0t80000gn/T/ipykernel_40693/4110285772.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabular_bm_df = tabular_bm_df.append(pd.DataFrame(final_report), ignore_index=True)


In [86]:
tabular_bm_df.tail()

,dataset,model,case,domain_cov,stats_cov,outliers_cov,missing_cov,ks_sim,tv_sim,corr_sim,contin_sim,sdv_quality_report,wass_dist,js_dist,new_row_synthesis
108,loan,ctgan_syn,hpo,0.97,0.99,0.40,1.0,0.91,0.98,0.96,0.93,0.94,0.03,0.17,1.0
109,loan,tvae_syn,hpo,0.95,0.98,0.35,1.0,0.93,0.98,0.96,0.92,0.94,0.02,0.19,1.0
110,loan,arf_syn,hpo,1.00,0.99,0.46,1.0,0.90,0.99,0.98,0.93,0.94,0.02,0.14,1.0
111,loan,ddpm_syn,hpo,1.00,0.94,0.45,1.0,0.95,0.97,0.98,0.88,0.93,0.03,0.10,1.0
112,loan,rtvae_syn,hpo,0.87,0.97,0.35,1.0,0.87,0.98,0.94,0.81,0.88,0.04,0.47,1.0


In [229]:


tabular_bm_df[tabular_bm_df["model"] == "ctgan_sdv"].style.highlight_max(color='#ffb6c1')


,dataset,model,case,domain_cov,stats_cov,outliers_cov,missing_cov,ks_sim,tv_sim,corr_sim,contin_sim,sdv_quality_report,wass_dist,js_dist,new_row_synthesis
1,adult,ctgan_sdv,tabular,0.870000,0.990000,0.790000,1.000000,0.870000,0.910000,0.980000,0.820000,0.870000,0.020000,0.220000,1.000000
19,child,ctgan_sdv,tabular,0.970000,-99.000000,-99.000000,0.980000,-99.000000,0.900000,nan,0.840000,0.870000,-99.000000,-99.000000,0.890000
30,covtype,ctgan_sdv,tabular,0.840000,0.970000,0.680000,1.000000,0.880000,0.960000,0.970000,0.910000,0.930000,0.040000,0.150000,1.000000
37,credit,ctgan_sdv,tabular,0.190000,1.000000,0.960000,1.000000,0.980000,1.000000,0.970000,0.550000,0.960000,0.000000,0.060000,0.920000
55,health_insurance,ctgan_sdv,tabular,0.980000,0.890000,0.350000,1.000000,0.750000,0.880000,0.890000,0.810000,0.820000,0.130000,0.240000,1.000000
68,intrusion,ctgan_sdv,tabular,0.670000,0.860000,0.360000,1.000000,0.770000,0.880000,0.920000,0.740000,0.800000,0.120000,0.220000,0.980000
75,drugs,ctgan_sdv,tabular,0.890000,0.960000,1.000000,0.950000,0.890000,0.920000,0.950000,0.830000,0.880000,0.060000,0.140000,-99.000000
86,loan,ctgan_sdv,tabular,0.970000,0.970000,0.370000,1.000000,0.850000,0.930000,0.910000,0.880000,0.890000,0.040000,0.150000,1.000000
100,pums,ctgan_sdv,tabular,0.820000,0.990000,0.670000,1.000000,0.880000,0.990000,0.990000,0.940000,0.950000,0.010000,0.190000,1.000000


In [97]:
import pandas as pd

# Assuming your DataFrame is named df and is already loaded with your data
# Example of loading the DataFrame (replace with your actual data loading method)
# df = pd.read_csv('your_data.csv')

def highlight_top3(s):
    '''
    Highlight the top 3 numerical values in each column with different colors.
    Skip non-numerical columns.
    '''
    if s.name in ['domain_cov', 'stats_cov', 'outliers_cov', 'missing_cov', 'ks_sim', 'tv_sim', 'corr_sim', 'contin_sim', 'sdv_quality_report', 'wass_dist', 'js_dist', 'new_row_synthesis']:
        top1 = s.nlargest(1)
        top2 = s.nlargest(2).drop(top1.index)
        top3 = s.nlargest(3).drop(top1.index).drop(top2.index)

        return ['background-color: gold' if v in top1.values else
                'background-color: pink' if v in top2.values else
                'background-color: grey' if v in top3.values else ''
                for v in s]
    else:
        return ['' for _ in s]  # Return empty styling for non-numerical columns

# df.style.apply(highlight_top3)



tabular_bm_df[tabular_bm_df["dataset"] == "adult"].style.apply(highlight_top3)


,dataset,model,case,domain_cov,stats_cov,outliers_cov,missing_cov,ks_sim,tv_sim,corr_sim,contin_sim,sdv_quality_report,wass_dist,js_dist,new_row_synthesis
0,adult,actgan_gretel,tabular,0.890000,0.990000,0.490000,1.000000,0.790000,0.920000,0.970000,0.870000,0.880000,0.020000,0.240000,1.000000
1,adult,ctgan_sdv,tabular,0.870000,0.990000,0.790000,1.000000,0.870000,0.910000,0.980000,0.820000,0.870000,0.020000,0.220000,1.000000
2,adult,tvae_sdv,tabular,0.790000,0.990000,0.680000,1.000000,0.880000,0.940000,0.970000,0.880000,0.910000,0.020000,0.130000,1.000000
3,adult,gaussian_copula_sdv,tabular,0.920000,0.950000,0.410000,1.000000,0.700000,0.800000,0.990000,0.730000,0.760000,0.070000,0.290000,1.000000
4,adult,ctgan_syn,tabular,0.910000,0.990000,0.490000,1.000000,0.760000,0.790000,0.980000,0.740000,0.780000,0.020000,0.340000,1.000000
5,adult,tvae_syn,tabular,0.860000,0.990000,0.650000,1.000000,0.810000,0.890000,0.990000,0.810000,0.840000,0.010000,0.300000,1.000000
6,adult,goggle_syn,tabular,0.330000,0.950000,0.270000,1.000000,0.620000,0.670000,0.970000,0.490000,0.590000,0.060000,0.540000,1.000000
7,adult,arf_syn,tabular,0.990000,1.000000,0.810000,1.000000,0.880000,0.910000,0.990000,0.870000,0.900000,0.010000,0.160000,1.000000
8,adult,ddpm_syn,tabular,0.990000,1.000000,0.680000,1.000000,0.980000,0.970000,0.980000,0.950000,0.970000,0.010000,0.060000,1.000000
9,adult,nflow_syn,tabular,0.970000,0.980000,0.700000,1.000000,0.890000,0.770000,0.970000,0.690000,0.770000,0.020000,0.230000,1.000000


In [41]:
tabular_bm_df.to_csv("tabular_benchmarks.csv")

In [ ]:
# pd.DataFrame(get_scores_df("adult"))

In [ ]:
# pd.DataFrame(get_scores_df("census"))

In [ ]:
# pd.DataFrame(get_scores_df("child"))

In [ ]:
# pd.DataFrame(get_scores_df("covtype"))

In [ ]:
# pd.DataFrame(get_scores_df("credit"))

In [ ]:
# pd.DataFrame(get_scores_df("intrusion"))

In [ ]:
# pd.DataFrame(get_scores_df("insurance"))

In [ ]:
# pd.DataFrame(get_scores_df("health_insurance"))

In [ ]:
# pd.DataFrame(get_scores_df("drugs"))

In [ ]:
# pd.DataFrame(get_scores_df("loan"))

In [ ]:
# pd.DataFrame(get_scores_df("pums"))

In [21]:
def get_ml_metrics(ml_efficacy_stats):
    # ml_efficacy_stats = data_dict["ml_efficacy"]
    ml_metrics = {}
    for classifier, scores in ml_efficacy_stats.items():
        if classifier == "timing":
            continue            
        ml_metrics[classifier] = round(abs(scores["synthetic_f1"] - scores["real_f1"]), 2)
    return ml_metrics

ML_CLASSIFICATION_TASK_DATASETS = [
    "adult", "census", "credit", "covtype", "loan", "intrusion"]
ML_REGRESSION_TASK_DATASETS = ["health_insurance"]
# ML_CLASSIFICATION_MODELS = ["adaboost", "decision_tree", "logistic", "mlp"]
# ML_REGRESSION_MODELS = ["linear", "mlp"]

In [88]:
ERROR_VAL  = -99
def get_ml_scores_df(exp_dataset, report, metric):
    for lib, models in ALL_TABULAR_MODELS.items():
        for model in models: 
            # corr_file = f"../metrics_out/tabular/{model}_{lib}/{exp_dataset}/{exp_dataset}_{model}_correlation.csv"
            metrics_file = f"../metrics_out/tabular/{model}_{lib}/{exp_dataset}/{exp_dataset}_{model}_metrics.json"
                        
            try:
                with open(metrics_file, 'r') as file:
                    metrics_scores = json.load(file)
            except Exception as e:
                # print(model, lib)
                # status = None
                # if lib in TABULAR_INCOMPLETE_JOBS and model in TABULAR_INCOMPLETE_JOBS[lib]:
                #     if exp_dataset in TABULAR_INCOMPLETE_JOBS[lib][model]:
                #         # print("Status: ",  TABULAR_INCOMPLETE_JOBS[lib][model][exp_dataset])
                #         # print("-")
                #         status = TABULAR_INCOMPLETE_JOBS[lib][model][exp_dataset]
                #     else:
                #         status = "Skipped"
                # else:
                #     status = "Skipped"
                #     # print(e)
                #     # print("Skipped")
                # for k in report.keys():
                #     if k not in ["model", "dataset", "case"]: 
                #         report[k].append(status)
                #     # print(final_report)
                continue
                
            report["dataset"].append(exp_dataset)
            report["model"].append(f"{model}_{lib}")
            
            for classifier, scores in metrics_scores["ml_efficacy"].items():
                if classifier == "timing":
                    continue            
                report[classifier].append(round(abs(scores[f"synthetic_{metric}"] - scores[f"real_{metric}"]), 2))
                # final_report_ml_cls["model"]["adaboost_classifier"].append()
                
    for k, v in report.items():
        print(k, len(v))
        
    
    return report

In [89]:
final_report_ml_cls = {
    "dataset": [],  
    "model": [],
    "adaboost_classification": [], 
    "decision_tree_classification": [], 
    "logistic_classification": [],
    "mlp_classification": [], 
    # "linear_regressor": [],
    # "mlp_regressor": []
}

ml_cls_metric = "f1"

final_report_ml_regress = {
    "dataset": [],  
    "model": [],
    "linear_regression": [], 
    "mlp_regression": []
}

ml_regress_metric = "r2"

In [90]:
# # "adult", "census", "credit", "covtype", "loan", "intrusion"]
pd.DataFrame(get_ml_scores_df("credit", final_report_ml_cls, ml_cls_metric))

dataset 8
model 8
adaboost_classification 8
decision_tree_classification 8
logistic_classification 8
mlp_classification 8


,dataset,model,adaboost_classification,decision_tree_classification,logistic_classification,mlp_classification
0,credit,actgan_gretel,0.71,0.33,0.09,0.78
1,credit,ctgan_sdv,0.00,0.00,0.00,0.00
2,credit,tvae_sdv,0.00,0.00,0.00,0.00
3,credit,gaussian_copula_sdv,0.19,0.11,0.06,0.14
4,credit,ctgan_syn,0.71,0.34,0.10,0.79
5,credit,tvae_syn,0.63,0.33,0.28,0.79
6,credit,arf_syn,0.07,0.00,0.05,0.08
7,credit,rtvae_syn,0.73,0.34,0.10,0.78


In [91]:
tabular_ml_bm_df = pd.DataFrame()
for exp_dataset in ML_CLASSIFICATION_TASK_DATASETS:
    final_ml_report = get_ml_scores_df(exp_dataset, final_report_ml_cls, ml_cls_metric)
    tabular_ml_bm_df = tabular_ml_bm_df.append(pd.DataFrame(final_ml_report), ignore_index=True)

dataset 22
model 22
adaboost_classification 22
decision_tree_classification 22
logistic_classification 22
mlp_classification 22
dataset 26
model 26
adaboost_classification 26
decision_tree_classification 26
logistic_classification 26
mlp_classification 26
dataset 34
model 34
adaboost_classification 34
decision_tree_classification 34
logistic_classification 34
mlp_classification 34
dataset 41
model 41
adaboost_classification 34
decision_tree_classification 34
logistic_classification 34
mlp_classification 34


/var/folders/3l/sc82d46d2_lbr0vhyw2tf0t80000gn/T/ipykernel_40693/2744727713.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabular_ml_bm_df = tabular_ml_bm_df.append(pd.DataFrame(final_ml_report), ignore_index=True)
/var/folders/3l/sc82d46d2_lbr0vhyw2tf0t80000gn/T/ipykernel_40693/2744727713.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabular_ml_bm_df = tabular_ml_bm_df.append(pd.DataFrame(final_ml_report), ignore_index=True)
/var/folders/3l/sc82d46d2_lbr0vhyw2tf0t80000gn/T/ipykernel_40693/2744727713.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabular_ml_bm_df = tabular_ml_bm_df.append(pd.DataFrame(final_ml_report), ignore_index=True)


ValueError: All arrays must be of the same length

In [38]:
final_report = get_ml_scores_df("health_insurance", final_report_ml_regress, ml_regress_metric)
pd.DataFrame(final_report)

dataset 13
model 13
linear_regression 14
mlp_regression 14


ValueError: All arrays must be of the same length

# Performance Evalutaion

In [ ]:
{
    "lib": "GRETEL_0.20.0",
    "modality": "tabular",
    "synthesizer": "actgan",
    "dataset": "adult",
    "num_rows": 26048,
    "num_cols": 16,
    "num_sampled_rows": 26048,
    "device": "GPU",
    "num_epochs": 300,
    "train_time_sec": 844.4202311038971,
    "sample_time_sec": 2.1354258060455322,
    "peak_memory_mb": 112.538464,
    "synthesizer_size": 37.958943,
    "synthetic_dataset_size_mb_deep": 16.898896,
    "train_dataset_size_mb_deep": 17.094568,
    "synthetic_dataset_size_mb": 3.334272,
    "train_dataset_size_mb": 3.542528
}

In [39]:
TABULAR_COMPLETED_JOBS = {
    "gretel": {
        "actgan": ["adult", "census", "child", "covtype", "credit", "insurance",
                   "intrusion", "drugs", "loan", "pums"]  # health_insurance
    },
    "sdv": {
        "gaussian_copula": ["adult", "census", "child", "covtype", "credit", "insurance",
                            "intrusion", "drugs", "loan", "pums", "health_insurance"],
        "ctgan": ["census", "child", "covtype", "credit",  # "insurance",
                  "intrusion", "drugs", "loan", "pums", "health_insurance"],
        # "adult",
        "tvae": ["adult", "census", "child", "covtype", "credit", "insurance",
                 "intrusion", "drugs", "loan", "pums", "health_insurance"]
    },
    "synthcity": {
        "ddpm": ["adult", "child", "covtype", "insurance",
                 "drugs", "loan", "health_insurance"],  # credit, intrusion, pums, census
        "arf": ["adult", "child", "covtype", "credit", "insurance",
                "intrusion", "drugs", "loan", "health_insurance"],  # census@, pums
        "nflow": ["adult", "child", "covtype", "credit", "insurance",
                  "intrusion", "drugs", "loan", "pums", "health_insurance"], #censusX
        "goggle": ["adult", "child", "insurance",
                   "drugs", "loan", "health_insurance"],  # intrusion, covtype, census, credit, pums
        "rtvae": ["adult", "child", "covtype", "credit", "insurance",
                  "drugs", "loan", "health_insurance"],  # census, pums, intrusion
        "tvae": ["adult", "child", "credit", "insurance",
                 "intrusion", "drugs", "loan", "health_insurance"],  # census, pums, covtype
        "ctgan": ["adult", "child", "covtype", "credit", "insurance",
                  "intrusion", "drugs", "loan", "health_insurance"]  # census, pums
    },
    "llm": {
        "great": ["adult", "health_insurance", "loan"]
    },
    "betterdata": {
        # "fake_companies"], #,
        "gan": ["adult", "census", "loan", "health_insurance"],
        "gan_dp": ["adult", "loan"]
    },
    "hpo_synthcity": {
        "arf": ["adult", "loan"],
        "ctgan": ["adult", "loan"],
        "ddpm": ["adult", "loan"],
        "rtvae": ["adult", "loan"],
        "tvae": ["adult", "loan"]
    }

}

In [184]:
ERROR_VAL  = -99
def get_execution_scores_df(exp_dataset, case="tabular"):
    
    final_report = {
        "dataset": [],
        "lib": [], 
        "model": [],
        "num_rows": [], 
        "num_cols": [], 
        "num_sampled_rows": [],
        "device": [],
        "num_epochs": [], 
        "train_time_sec": [],
        "sample_time_sec": [], 
        "peak_memory_mb": [], 
        "synthesizer_size": [],
        "synthetic_dataset_size_mb_deep": [], 
        "train_dataset_size_mb_deep": [], 
        "synthetic_dataset_size_mb": [],
        "train_dataset_size_mb": []
    }
    
    for lib, models in ALL_TABULAR_MODELS.items():
        
        # if case == "hpo" and lib!= "syn":
        #     continue

        for model in models: 
            
            BASE = f"final_outs/{lib}_tabular"
            execution_scores_path = f"../{BASE}/{model}/{exp_dataset}/{exp_dataset}_{model}_execution_scores.json"
            
            
            try:
                with open(execution_scores_path, 'r') as file:
                    execution_scores = json.load(file)
            except Exception as e:
                print(e)
                continue
                
                
                
            # corr_file = f"../metrics_out/{case}/{model}_{lib}/{exp_dataset}/{exp_dataset}_{model}_correlation.csv"
            # metrics_file = f"../metrics_out/{case}/{model}_{lib}/{exp_dataset}/{exp_dataset}_{model}_metrics.json"
            final_report["model"].append(f"{model}_{lib}")
            for k, v in final_report.items():
                if k in ["model"]: continue # "modality", "synthesizer"
                
                if k not in execution_scores:
                    final_report[k].append("-99")
                    continue
                final_report[k].append(execution_scores[k])
                    
# execution_scores
    # for k, v in final_report.items():
    #     print(k, len(v))
                    
    return final_report

In [185]:
# execution_scores_df = pd.DataFrame(get_scores_df("adult", case="tabular"))
# execution_scores_df.shape
# execution_scores_df


execution_scores_df = pd.DataFrame()
for exp_dataset in ALL_TABULAR_DATASETS:
    final_exe_report = get_execution_scores_df(exp_dataset, case="tabular")
    execution_scores_df = execution_scores_df.append(pd.DataFrame(final_exe_report), ignore_index=True)

[Errno 2] No such file or directory: '../final_outs/betterdata_tabular/gan/adult/adult_gan_execution_scores.json'
[Errno 2] No such file or directory: '../final_outs/betterdata_tabular/gan_dp/adult/adult_gan_dp_execution_scores.json'
[Errno 2] No such file or directory: '../final_outs/syn_tabular/ctgan/census/census_ctgan_execution_scores.json'
[Errno 2] No such file or directory: '../final_outs/syn_tabular/tvae/census/census_tvae_execution_scores.json'
[Errno 2] No such file or directory: '../final_outs/syn_tabular/goggle/census/census_goggle_execution_scores.json'
[Errno 2] No such file or directory: '../final_outs/syn_tabular/arf/census/census_arf_execution_scores.json'
[Errno 2] No such file or directory: '../final_outs/syn_tabular/ddpm/census/census_ddpm_execution_scores.json'
[Errno 2] No such file or directory: '../final_outs/syn_tabular/nflow/census/census_nflow_execution_scores.json'
[Errno 2] No such file or directory: '../final_outs/syn_tabular/rtvae/census/census_rtvae_exec

/var/folders/3l/sc82d46d2_lbr0vhyw2tf0t80000gn/T/ipykernel_40693/3890925574.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  execution_scores_df = execution_scores_df.append(pd.DataFrame(final_exe_report), ignore_index=True)
/var/folders/3l/sc82d46d2_lbr0vhyw2tf0t80000gn/T/ipykernel_40693/3890925574.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  execution_scores_df = execution_scores_df.append(pd.DataFrame(final_exe_report), ignore_index=True)
/var/folders/3l/sc82d46d2_lbr0vhyw2tf0t80000gn/T/ipykernel_40693/3890925574.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  execution_scores_df = execution_scores_df.append(pd.DataFrame(final_exe_report), ignore_index=True)
/var/folders/3l/sc82d46d2_lbr0vhyw2tf0t80000gn/T/

In [186]:
execution_scores_df

,dataset,lib,model,num_rows,num_cols,num_sampled_rows,device,num_epochs,train_time_sec,sample_time_sec,peak_memory_mb,synthesizer_size,synthetic_dataset_size_mb_deep,train_dataset_size_mb_deep,synthetic_dataset_size_mb,train_dataset_size_mb
0,adult,GRETEL_0.20.0,actgan_gretel,26048,16,26048,GPU,300,844.420231,2.135426,112.538464,37.958943,16.898896,17.094568,3.334272,3.542528
1,adult,SDV_1.2.1,ctgan_sdv,26048,15,32561,CPU,300,1019.838562,2.317747,101.742050,35.808308,20.880857,16.886184,3.907448,3.334144
2,adult,SDV_1.2.1,tvae_sdv,26048,15,32561,CPU,300,378.850136,0.980367,101.740078,0.417060,20.859236,16.886184,3.907448,3.334144
3,adult,SDV_1.2.1,gaussian_copula_sdv,26048,15,32561,CPU,0,11.544246,1.632146,32.554806,0.135811,20.769536,16.886184,3.907448,3.334144
4,adult,synthcity==0.2.9,ctgan_syn,26048,15,32561,cpu,300,5387.746336,9.032208,161.648593,38.916708,21.105475,16.886184,4.167808,3.334144
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,loan,GREAT,great_llm,5000,14,5000,GPU,100,2870.947590,35.642844,203.695731,327.700000,-99,-99,-99,-99
94,pums,GRETEL_0.20.0,actgan_gretel,828161,36,828161,GPU,300,50401.469090,198.961060,9927.379896,3308.896283,284.691334,291.35225,238.510496,245.135656
95,pums,SDV_1.2.1,ctgan_sdv,828161,36,-99,GPU,300,34275.731180,190.307225,8131.216867,2639.754827,355.908334,291.35225,298.138304,245.135656
96,pums,SDV_1.2.1,tvae_sdv,828161,36,-99,GPU,300,22887.041172,153.788952,8131.214587,0.997506,355.923838,291.35225,298.138304,245.135656


In [230]:
def highlight_top3(s):
    '''
    Highlight the top 3 numerical values in each column with different colors.
    Skip non-numerical columns.
    '''
    if s.name not in ["dataset", "lib", "model", "device", "num_sampled_rows", "synthetic_dataset_size_mb_deep", 
                     "train_dataset_size_mb_deep", "synthetic_dataset_size_mb", "train_dataset_size_mb"]:
    # in ['domain_cov', 'stats_cov', 'outliers_cov', 'missing_cov', 'ks_sim', 'tv_sim', 'corr_sim', 'contin_sim', 'sdv_quality_report', 'wass_dist', 'js_dist', 'new_row_synthesis']:
        print(s.name)
        top1 = s.nlargest(1)
        top2 = s.nlargest(2).drop(top1.index)
        top3 = s.nlargest(3).drop(top1.index).drop(top2.index)

        return ['background-color: gold' if v in top1.values else
                'background-color: pink' if v in top2.values else
                'background-color: grey' if v in top3.values else ''
                for v in s]
    else:
        return ['' for _ in s]  # Return empty styling for non-numerical columns

# df.style.apply(highlight_top3)

execution_scores_df[execution_scores_df["model"] == "ctgan_sdv"].style.apply(highlight_top3)

num_rows
num_cols
num_epochs
train_time_sec
sample_time_sec
peak_memory_mb
synthesizer_size


,dataset,lib,model,num_rows,num_cols,num_sampled_rows,device,num_epochs,train_time_sec,sample_time_sec,peak_memory_mb,synthesizer_size,synthetic_dataset_size_mb_deep,train_dataset_size_mb_deep,synthetic_dataset_size_mb,train_dataset_size_mb
1,adult,SDV_1.2.1,ctgan_sdv,26048,15,32561,CPU,300,1019.838562,2.317747,101.742050,35.808308,20.880857,16.886184,3.907448,3.334144
13,census,SDV_1.2.1,ctgan_sdv,239428,41,299285,CPU,300,38185.008448,38.364032,2861.407736,986.504407,646.948582,519.801752,98.165608,80.447808
17,child,SDV_1.2.1,ctgan_sdv,16000,20,20000,CPU,300,534.051591,1.355828,29.154026,11.255035,24.530349,19.697801,3.200128,2.688000
28,covtype,SDV_1.2.1,ctgan_sdv,464809,55,581012,CPU,300,34465.168423,101.741135,2795.060875,868.571270,255.645408,208.234432,255.645408,208.234432
35,credit,SDV_1.2.1,ctgan_sdv,227845,30,284807,CPU,300,11258.270308,18.948731,1035.224630,327.824054,68.353808,56.505560,68.353808,56.505560
53,health_insurance,SDV_1.2.1,ctgan_sdv,1070,7,1338,CPU,300,24.414932,0.189142,2.309114,1.107998,0.293731,0.243080,0.075056,0.068480
65,intrusion,SDV_1.2.1,ctgan_sdv,395216,41,494021,CPU,300,29959.334958,72.893574,2176.398364,675.488391,266.429332,216.029772,162.039016,132.792576
72,drugs,SDV_1.2.1,ctgan_sdv,6107,40,7634,CPU,300,63133.236859,7.662645,1482.101211,947.764680,13.947227,10.922882,2.443008,2.003096
83,loan,SDV_1.2.1,ctgan_sdv,4000,14,5000,CPU,300,79.544001,0.417012,10.730754,4.088130,0.560128,0.480000,0.560128,0.480000
95,pums,SDV_1.2.1,ctgan_sdv,828161,36,-99,GPU,300,34275.731180,190.307225,8131.216867,2639.754827,355.908334,291.352250,298.138304,245.135656


In [ ]:
# Sequential

,dataset,lib,model,num_rows,num_cols,num_sampled_rows,device,num_epochs,train_time_sec,sample_time_sec,peak_memory_mb,synthesizer_size,synthetic_dataset_size_mb_deep,train_dataset_size_mb_deep,synthetic_dataset_size_mb,train_dataset_size_mb


# HyperImpute

In [ ]:
ERROR_VAL  = -99

# /Users/anshusingh/DPPCC/whitespace/benchmarking-synthetic-data-generators/final_outs/hyperimpute_tabular/drugs/ctgan_missforest/drugs/drugs_ctgan_synthetic_data.csv
/Users/anshusingh/DPPCC/whitespace/benchmarking-synthetic-data-generators/final_outs/hyperimpute_tabular/drugs/ctgan_missforest/drugs/drugs_imputed_data.csv
def get_scores_df(exp_dataset, models_list, case="tabular"):
    
    final_report = {
        "dataset": [], #drugs
        "model": [], # ctgan
        "case": [], # hyperimpute
        "domain_cov": [], 
        "stats_cov": [], 
        "outliers_cov": [],
        "missing_cov": [],
        "ks_sim": [], 
        "tv_sim": [],
        "corr_sim": [], 
        "contin_sim": [], 
        "sdv_quality_report": [],
        "wass_dist": [], 
        "js_dist": [], 
        "new_row_synthesis": []
    }

    for lib, models in models_list.items():
        
        if case == "hpo" and lib!= "syn":
            continue
        for model in models: 
                
            corr_file = f"../metrics_out/{case}/{model}_{lib}/{exp_dataset}/{exp_dataset}_{model}_correlation.csv"
            metrics_file = f"../metrics_out/{case}/{model}_{lib}/{exp_dataset}/{exp_dataset}_{model}_metrics.json"
                        
            try:
                with open(metrics_file, 'r') as file:
                    metrics_scores = json.load(file)
                corr_df = pd.read_csv(corr_file)
                print(metrics_file)
            except Exception as e:
                print(e)
                # print(model, lib)
                # status = None
                # if lib in TABULAR_INCOMPLETE_JOBS and model in TABULAR_INCOMPLETE_JOBS[lib]:
                #     if exp_dataset in TABULAR_INCOMPLETE_JOBS[lib][model]:
                #         # print("Status: ",  TABULAR_INCOMPLETE_JOBS[lib][model][exp_dataset])
                #         # print("-")
                #         status = TABULAR_INCOMPLETE_JOBS[lib][model][exp_dataset]
                #     else:
                #         status = -99 #"Skipped"
                # else:
                #     status =  -99 #"Skipped"
                #     # print(e)
                #     # print("Skipped")
                # for k in final_report.keys():
                #     if k not in ["model", "dataset", "case"]: 
                        # final_report[k].append(-99) #.append(status)
                    # print(final_report)
                continue
            
            final_report["model"].append(f"{model}_{lib}")
            final_report["dataset"].append(exp_dataset)
            final_report["case"].append(case)
            
            # print(final_report) 
            
            if metrics_scores["coverage"]["domain_coverage"].values():
                domain_coverage_mean = round(stats.mean(metrics_scores["coverage"]["domain_coverage"].values()), 2)
                final_report["domain_cov"].append(domain_coverage_mean)
            else:
                # Handle the case for empty list, e.g., append a default value or skip
                final_report["domain_cov"].append(ERROR_VAL)  # or any other default value
            
            
            if metrics_scores["coverage"]["missing_values_coverage"].values():
                missing_values_coverage_mean = round(stats.mean(metrics_scores["coverage"]["missing_values_coverage"].values()), 2)
                final_report["missing_cov"].append(missing_values_coverage_mean)
            else:
                # Handle the case for empty list, e.g., append a default value or skip
                final_report["missing_cov"].append(ERROR_VAL)  # or any other default value
            
            
            if metrics_scores["coverage"]["outlier_coverage"].values():
                outlier_coverage_mean = round(stats.mean(metrics_scores["coverage"]["outlier_coverage"].values()), 2)
                final_report["outliers_cov"].append(outlier_coverage_mean)
            else:
                # Handle the case for empty list, e.g., append a default value or skip
                final_report["outliers_cov"].append(ERROR_VAL)  # or any other default value

            # final_report["outliers_cov"].append(round(stats.mean(metrics_scores["coverage"]["outlier_coverage"].values()), 2))

            # Initialize total and count variables
            total_average = 0
            count = 0
            # Calculate the average of mean, median, and std for each column
            try:
                for c, s in metrics_scores["similarity"]["statistic"].items():
                    column_average = (s["mean"] + s["median"] + s["std"]) / 3
                    total_average += column_average
                    count += 1
                # Calculate the overall average
                overall_average = total_average / count if count > 0 else ERROR_VAL
                final_report["stats_cov"].append(round(overall_average, 2))
            except:
                final_report["stats_cov"].append(ERROR_VAL)

            ks_scores = []
            tv_scores = []
            for key, value in metrics_scores["sdv_quality_report"]["distribution"].items():
                if value["metric"] == "KSComplement":
                    ks_scores.append(value["score"])
                elif value["metric"] == "TVComplement":
                    tv_scores.append(value["score"])

            # Calculating averages
            ks_average = stats.mean(ks_scores) if ks_scores else ERROR_VAL
            tv_average = stats.mean(tv_scores) if tv_scores else ERROR_VAL
            final_report["ks_sim"].append(round(ks_average, 2))
            final_report["tv_sim"].append(round(tv_average, 2))


            # contingency_average, correlation_average
            contingency_average = corr_df[corr_df['Metric'] == 'ContingencySimilarity']["Score"].mean()
            correlation_average = corr_df[corr_df['Metric'] == 'CorrelationSimilarity']["Score"].mean()

            final_report["contin_sim"].append(round(contingency_average, 2))
            final_report["corr_sim"].append(round(correlation_average, 2))

            final_report["sdv_quality_report"].append(round(metrics_scores["sdv_quality_report"]["score"], 2))
            
            if metrics_scores["similarity"]["js_distance"].values():
                js_dist_mean = round(stats.mean(metrics_scores["similarity"]["js_distance"].values()), 2)
                final_report["js_dist"].append(js_dist_mean)
            else:
                # Handle the case for empty list, e.g., append a default value or skip
                final_report["js_dist"].append(ERROR_VAL)  # or any other default value
                
            if metrics_scores["similarity"]["wass_distance"].values():
                wass_dist_mean = round(stats.mean(metrics_scores["similarity"]["wass_distance"].values()), 2)
                final_report["wass_dist"].append(wass_dist_mean)
            else:
                # Handle the case for empty list, e.g., append a default value or skip
                final_report["wass_dist"].append(ERROR_VAL)  # or any other default value

            # # final_report["js_dist"].append(round(stats.mean(metrics_scores["similarity"]["js_distance"].values()), 2))
            # final_report["wass_dist"].append(round(stats.mean(metrics_scores["similarity"]["wass_distance"].values()), 2))
            
            if "score" in metrics_scores["privacy"]["new_row_synthesis"]:
                final_report["new_row_synthesis"].append(round(metrics_scores["privacy"]["new_row_synthesis"]["score"], 2))
            else:
                final_report["new_row_synthesis"].append(ERROR_VAL)
            
    return final_report